In [237]:
import pandas as pd
data = pd.read_csv('/Users/tamasmakos/dev/k-monitor/Export_KMONITOR.csv')
data = data.sample(2000)

In [238]:
data['Nyertes ajánlattevő neve'].isnull().sum()/len(data['Nyertes ajánlattevő neve'])

0.228

In [239]:
must_and_nice_to_have_columns = [
    'Eljárás EKR azonosító',
    'Ajánlatkérő szervezet neve',
    'Ajánlatkérő nemzeti azonosítószáma',
    'Ajánlatkérő szervezet típusa',
    'Ajánlatkérő szervezet főtevékenysége',
    'Eljárásrend',
    'Eljárás fajtája',
    'Eljárás tárgya',
    'Szerződés típusa',
    'A beszerzés végleges összértéke',
    'A beszerzés végleges összértéke pénznem',
    'Teljesítés helye NUTS-kód(ok)',
    'Szerződés megkötésének dátuma',
    'Beérkezett ajánlatok száma',
    'Nyertes ajánlattevő adószáma (adóazonosító jele)',
    'Nyertes ajánlattevő neve',
    'Eljárás tárgya',
    'Fő CPV-kód(ok)',
    'További CPV-kód(ok)',
    'A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos',
    'Szerződés/rész odaítélésre került',
    'Nyertes ajánlattevő kkv',
    'Szerződés megkötésének dátuma',
    'Hirdetmény közzétételének dátuma',
    'Minőségi kritérium alkalmazásra került',
    'Költség kritérium alkalmazásra került',
    'Ár kritérium alkalmazásra került',
    'A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos',
    'Szerződés/rész odaítélésre került',
    'Alvállalkozók igénybevétele a szerződés teljesítéséhez',
    'Alvállalkozó(k) megnevezése, adószáma'
]


In [206]:
data = data[must_and_nice_to_have_columns]

In [240]:
data['Nyertes ajánlattevő neve'].isnull().sum()/len(data['Nyertes ajánlattevő neve'])

0.228

In [241]:
def process_contracts(df, issuer_name_col, issuer_id_col, winner_name_col, winner_id_col):
    # Reset the index of the DataFrame to ensure uniqueness
    df = df.reset_index(drop=True)

    def safe_strip(value):
        """ Safely strip strings, handling NaN values """
        return value.strip() if isinstance(value, str) else value

    # Creating a new DataFrame to hold the expanded data
    expanded_data = []

    for _, row in df.iterrows():
        issuer_names = row[issuer_name_col].split('|') if pd.notna(row[issuer_name_col]) else ['']
        issuer_ids = row[issuer_id_col].split('|') if pd.notna(row[issuer_id_col]) else ['']
        winner_names = row[winner_name_col].split('|') if pd.notna(row[winner_name_col]) else ['']
        winner_ids = row[winner_id_col].split('|') if pd.notna(row[winner_id_col]) else ['']

        for issuer_name, issuer_id in zip(issuer_names, issuer_ids):
            for winner_name, winner_id in zip(winner_names, winner_ids):
                if safe_strip(issuer_name) and safe_strip(winner_name):
                    # Convert row to dictionary to avoid copying the index
                    new_row = row.to_dict()
                    new_row[issuer_name_col] = safe_strip(issuer_name)
                    new_row[issuer_id_col] = safe_strip(issuer_id)
                    new_row[winner_name_col] = safe_strip(winner_name)
                    new_row[winner_id_col] = safe_strip(winner_id)
                    expanded_data.append(new_row)

    return pd.DataFrame(expanded_data)

expanded_df = process_contracts(
    data, 
    'Ajánlatkérő szervezet neve', 
    'Ajánlatkérő nemzeti azonosítószáma', 
    'Nyertes ajánlattevő neve', 
    'Nyertes ajánlattevő adószáma (adóazonosító jele)'
)


In [243]:
expanded_df['Nyertes ajánlattevő neve'].isnull().sum()/len(expanded_df['Nyertes ajánlattevő neve'])

0.0

In [208]:
import re

def general_text_cleaning(df, column_name):
    """
    Cleans the text in the specified column of a DataFrame.

    Parameters:
    df (DataFrame): The dataframe to be processed.
    column_name (str): The name of the column to clean.

    Returns:
    DataFrame: The dataframe with cleaned text in the specified column.
    """

    # Making a copy of the dataframe to avoid modifying the original data
    cleaned_df = df.copy()

    # Function to clean individual text entries
    def clean_text(text):
        # Ensuring that the input is a string
        if not isinstance(text, str):
            if pd.isna(text):
                return "Ismeretlen"  # Replace NaN with "Ismeretlen"
            else:
                return str(text)  # Convert non-string, non-NaN values to string

        # Proceed with cleaning
        text = text.strip()  # Trimming white spaces
        text = re.sub(r'\s+', ' ', text)  # Normalizing whitespaces
        text = text.title()  # Standardizing capitalization
        text = re.sub(r'[^A-Za-z0-9 ÁÉÍÓÖŐÚÜŰáéíóöőúüű,.]', '', text)  # Removing special characters

        return text

    # Applying the cleaning function to the specified column
    cleaned_df[column_name] = cleaned_df[column_name].apply(clean_text)

    return cleaned_df

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd

def standardize_entries_with_log_and_similarity_enhanced(dataframe, column_name, threshold=0.85):
    dataframe['unique_id'] = range(len(dataframe))
    unique_values = dataframe[[column_name, 'unique_id']].drop_duplicates(subset=[column_name])

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(unique_values[column_name])

    cosine_similarities = linear_kernel(tfidf_matrix)  # More efficient than cosine_similarity

    standardized_names = {}
    change_log_data = []

    # Iterate only over the upper triangle of the similarity matrix to avoid redundant comparisons
    for i in range(len(cosine_similarities)):
        for j in range(i + 1, len(cosine_similarities)):
            if cosine_similarities[i, j] >= threshold:
                entry_i = unique_values.iloc[i][column_name]
                entry_j = unique_values.iloc[j][column_name]

                # Decide which entry to standardize based on occurrence
                if entry_i in standardized_names:
                    standardized_names[entry_j] = standardized_names[entry_i]
                else:
                    standardized_names[entry_i] = entry_j
                    standardized_names[entry_j] = entry_j

                # Log changes
                change_log_data.append({
                    'Unique ID': unique_values.iloc[j]['unique_id'],
                    'Original Value': entry_j,
                    'Match': entry_i,
                    'New Value': standardized_names[entry_j],
                    'Similarity Score': cosine_similarities[i][j]
                })

    dataframe[column_name] = dataframe[column_name].map(standardized_names)
    change_log_df = pd.DataFrame(change_log_data)

    return dataframe, change_log_df


In [245]:
def standardize_company_type(company_type):
    """
    Standardize Hungarian company types based on common names, abbreviations, and misspellings.
    
    Args:
    company_type (str): The company type string to standardize.
    
    Returns:
    str: The standardized abbreviation of the company type.
    """
    try:
      lower_type = company_type.lower()

      # Check for "Korlátolt felelősségű társaság" and its abbreviations
      if "korlátolt" in lower_type and "felelősségű" in lower_type and "társaság" in lower_type or lower_type.replace(".", "") == "kft":
          return "Kft."

      # Check for "Részvénytársaság" and its abbreviations
      elif "részvénytársaság" in lower_type or "részvény" in lower_type or lower_type.replace(".", "") == "rt":
          return "Rt."

      # Check for "Zártkörűen működő részvénytársaság" and its abbreviations
      elif "zártkörűen" in lower_type and "működő" in lower_type and "részvénytársaság" in lower_type or lower_type.replace(".", "") == "zrt":
          return "Zrt."

      # Check for "egyéni vállalkozó" and its abbreviations
      elif "egyéni" in lower_type and "vállalkozó" in lower_type or lower_type.replace(".", "") in ["ev", "e v"]:
          return "EV"

      # Check for "Betéti társaság" and its abbreviations
      elif "betéti" in lower_type and "társaság" in lower_type or lower_type.replace(".", "") == "bt":
          return "Bt."

      # Check for "Szövetkezet" and its abbreviations
      elif "szövetkezet" in lower_type or lower_type.replace(".", "") == "sz":
          return "Sz."

      # Check for "Közkereseti társaság" and its abbreviations
      elif "közkereseti" in lower_type and "társaság" in lower_type or lower_type.replace(".", "") == "kkt":
          return "Kkt."

    except AttributeError:
      pass

    # Return original if no match found
    return company_type

# Reapply the updated function to the dataset
expanded_df['Ajánlatkérő szervezet neve'] = expanded_df['Ajánlatkérő szervezet neve'].apply(standardize_company_type)
expanded_df['Nyertes ajánlattevő neve'] = expanded_df['Nyertes ajánlattevő neve'].apply(standardize_company_type)

In [211]:
text_cleaning = ['Ajánlatkérő szervezet neve', 'Nyertes ajánlattevő neve']

In [212]:
import pandas as pd
import os
import ctypes
import gc

# Looping through the list of column names
for column in text_cleaning:
    # Initial Step: Replace all NaN values with "Ismeretlen"
    expanded_df[column] = expanded_df[column].fillna("Ismeretlen")

    # Step 1: Clean the column using general_text_cleaning
    cleaned_text_data = general_text_cleaning(expanded_df, column)

    # Step 2: Standardize entries and generate change_log_df
    data, change_log_df = standardize_entries_with_log_and_similarity_enhanced(cleaned_text_data, column, 0.70)

    # Step 3: Save the change_log_df to a CSV file
    change_log_filename = f'{column}_change_log.csv'
    change_log_df.to_csv(change_log_filename, index=False)

    print(f"Processed and saved change log for column '{column}' in file '{change_log_filename}'.")

print("All specified columns have been processed.")


Processed and saved change log for column 'Ajánlatkérő szervezet neve' in file 'Ajánlatkérő szervezet neve_change_log.csv'.
Processed and saved change log for column 'Nyertes ajánlattevő neve' in file 'Nyertes ajánlattevő neve_change_log.csv'.
All specified columns have been processed.


In [219]:
data['Nyertes ajánlattevő neve'].value_counts().sort_values(ascending=False)

# Plotting the distribution of the number of contracts per company
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
counts = data['Nyertes ajánlattevő neve'].value_counts().sort_values(ascending=False)
plt.hist(counts, bins=1000)
plt.xlabel('Number of contracts per company')
plt.ylabel('Frequency')
plt.yscale('log')
plt.title('Distribution of the number of contracts per company')
plt.show()


New Land Media Reklám, Szolgáltató És Kereskedelmi Korlátolt Felelősségű Társaság             14
Rk Tech Kereskedelmi És Szolgáltató Korlátolt Felelősségű Társaság                            10
Mészáros És Mészáros Ipari, Kereskedelmi És Szolgáltató Zártkörűen Működő Részvénytársaság     8
Premier G. Med Cardio Kft.                                                                     8
Biotech Hungary Kft.                                                                           7
Strabag Általános Építő Korlátolt Felelősségű Társaság                                         7
Greiner BioOne Hungary Kft                                                                     5
Vwr International Kft                                                                          5
BioScience Kft.                                                                                5
Allied Water Solutions Cee Kft.                                                                4
P2M Informatika Szolgáltató Ko

In [227]:
print(data.head())


  Eljárás EKR azonosító                         Ajánlatkérő szervezet neve  \
0       EKR001376162019     Nemzeti Biodiverzitás- és Génmegőrzési Központ   
1       EKR000211952019  MÁV Magyar Államvasutak Zártkörűen Működő Rész...   
2       EKR000211952019  MÁV Magyar Államvasutak Zártkörűen Működő Rész...   
3       EKR000211952019  MÁV Magyar Államvasutak Zártkörűen Működő Rész...   
4       EKR000211952019  MÁV Szolgáltató Központ Zártkörűen Működő Rész...   

  Ajánlatkérő nemzeti azonosítószáma Ajánlatkérő szervezet típusa  \
0                        15779935213          N - Központi szintű   
1                     EKRSZ_35803375             U - Egyéb típus:   
2                     EKRSZ_35803375             U - Egyéb típus:   
3                     EKRSZ_35803375             U - Egyéb típus:   
4                     EKRSZ_54810576             U - Egyéb típus:   

  Ajánlatkérő szervezet főtevékenysége          Eljárásrend  \
0              KL - Egyéb tevékenység:  Nemzeti eljár

In [230]:
# Creating a bipartite graph of companies and contracts and A beszerzés végleges összértéke as weight
import networkx as nx

# Creating a new graph
G = nx.Graph()

# Adding nodes with the node attribute "bipartite"
G.add_nodes_from(data['Ajánlatkérő szervezet neve'], bipartite=0)
G.add_nodes_from(data['Nyertes ajánlattevő neve'], bipartite=1)

# Adding edges with the edge attribute "weight"
for _, row in data.iterrows():
    G.add_edge(row['Ajánlatkérő szervezet neve'], row['Nyertes ajánlattevő neve'], weight=row['A beszerzés végleges összértéke'])

In [236]:
from networkx.drawing.nx_agraph import graphviz_layout

# Calculating the frequency of each node
node_sizes = [G.degree(node) * 100 for node in G]

# Drawing the graph
plt.figure(figsize=(12, 12))
pos = graphviz_layout(G, prog='neato')
nx.draw(G, pos, with_labels=True, node_size=node_sizes, edge_color='gray', alpha=0.4, linewidths=0.5)
plt.title('Bipartite Graph of Companies and Contracts')
plt.show()

ImportError: requires pygraphviz http://pygraphviz.github.io/

<Figure size 1200x1200 with 0 Axes>

# 

In [200]:
list(data.columns)

['Eljárás EKR azonosító',
 'Ajánlatkérő szervezet neve',
 'Ajánlatkérő nemzeti azonosítószáma',
 'Ajánlatkérő szervezet típusa',
 'Ajánlatkérő szervezet főtevékenysége',
 'Eljárásrend',
 'Eljárás fajtája',
 'Eljárás tárgya',
 'Szerződés típusa',
 'A beszerzés végleges összértéke',
 'A beszerzés végleges összértéke pénznem',
 'Teljesítés helye NUTS-kód(ok)',
 'Szerződés megkötésének dátuma',
 'Beérkezett ajánlatok száma',
 'Nyertes ajánlattevő adószáma (adóazonosító jele)',
 'Nyertes ajánlattevő neve',
 'Fő CPV-kód(ok)',
 'További CPV-kód(ok)',
 'A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos',
 'Szerződés/rész odaítélésre került',
 'Nyertes ajánlattevő kkv',
 'Hirdetmény közzétételének dátuma',
 'Minőségi kritérium alkalmazásra került',
 'Költség kritérium alkalmazásra került',
 'Ár kritérium alkalmazásra került',
 'Alvállalkozók igénybevétele a szerződés teljesítéséhez',
 'Alvállalkozó(k) megnevezése, adószáma',
 'unique_id']

In [202]:
import requests

API_URL = "https://px843j3fuq5o3nz6.us-east-1.aws.endpoints.huggingface.cloud"
headers = {
	"Authorization": "Bearer XXXXXX",
	"Content-Type": "application/json"
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "Mit jelent ez a közbeszerzési eljárásnak a tárgy: KR heli. jav. KM-ból hibafeltárás, hibajavítás.",
    "parameters": {
        "max_new_tokens": 100,
		"repetition_penalty": 0.4
    }
	
})

print(output)

[{'generated_text': ' KR heli. jav. KM-ból hibafeltárás, hibajavítás. KR heli. jav. KM-ból hibafeltárás, hibajavítás. KR heli. jav. KM-ból hibafeltárás, hibajavítás. KR heli. jav. KM-ból hibafeltárás'}]
